<a href="https://colab.research.google.com/github/Ian-Ibrahim/SU-Machine-learning-1222785/blob/main/26_6_2023_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1=nn.Conv2d(3,16,kernel_size=3,stride=1,padding=1)
    self.relu=nn.ReLU()
    self.maxpool=nn.MaxPool2d(kernel_size=2,stride=2)
    self.convl2=nn.Conv2d(16,32,kernel_size=3,stride=1,padding=1)
    self.fc=nn.Linear(32*8*8,10)

  def forward(self,x):
    x=self.conv1(x)
    x=self.relu(x)
    x=self.maxpool(x)

    x=self.convl2(x)
    x=self.relu(x)
    x=self.maxpool(x)

    x=x.view(x.size(0),-1)
    x=self.fc(x)

    return x

In [ ]:
def train(model,train_loader,criterion,optimizer,device):
  model.train()
  for images,labels in train_loader:
    images,labels=images.to(device),labels.to(device)
    optimizer.zero_grad()

    outputs=model(images)
    loss=criterion(outputs,labels)
    loss.backward()

    optimizer.step()
def evaluate(model,test_loader,device):
  model.eval()
  correct=0
  total=0

  with torch.no_grad():
    for images,labels in test_loader:
      images,labels=images.to(device),labels.to(device)
      outputs=model(images)
      # _,varname is a ghost variable helps with effiency
      _,predicted=torch.max(outputs.data,1)
      total+=labels.size(0)
      correct+=(predicted==labels).sum().item()

    acc=100*correct/total
    return acc

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# CIFA-10

# typically batch size is in multiples of 4
# num_workers is the number of CPUS



# CIFAR-10
train_set = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:02<00:00, 79544908.78it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# cuda
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
device

device(type='cuda')

In [ ]:
model=CNN().to(device)
criterion=nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(),lr=0.01,momentum=0.9,)

In [ ]:
for epoch in range(2):
  train(model,train_loader,criterion,optimizer,device)
  acc=evaluate(model,test_loader,device)
  print("Epoch: ",epoch+1)
  print("Accuracy: ",acc)

Epoch:  1
Accuracy:  56.41
Epoch:  2
Accuracy:  63.35
